In [12]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql import functions as F # import when, lit, expr
# from pyspark.sql.functions import col as sparkcol



In [2]:
spark = SparkSession.builder.appName('spark_test').getOrCreate()

sparkContext = spark.sparkContext

21/07/18 17:00:30 WARN Utils: Your hostname, MIM resolves to a loopback address: 127.0.1.1; using 192.168.0.155 instead (on interface wlp2s0)
21/07/18 17:00:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/18 17:00:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])

In [4]:
df = spark.createDataFrame(data=data2,schema=schema)


In [5]:
df.show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+



In [6]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']

numeric_features

['salary']

In [83]:
# Example:
# for column_name in numeric_features:
#     df_clean = df_clean.withColumn(column_name, F.when(F.col(column_name) == -1, None).otherwise(F.col(column_name)))

# df_clean.show()

def df_replaced_values(df, value, numeric_features, quantile=None):
     if quantile != None:
          for column_name in numeric_features:
               # replacement value is the median...
               quant = df.approxQuantile(column_name, [0.5], 0)
               df_replaced = df.withColumn(column_name, F.when(F.col(column_name) == value[0], quant[0]).otherwise(F.col(column_name)))
               return df_replaced
          
     else:
          for column_name in numeric_features:
               df_replaced = df.withColumn(column_name, F.when(F.col(column_name) == None, value[1])\
                    .when(F.col(column_name) == value[0], value[1])\
                    .otherwise(F.col(column_name)))
               return df_replaced

In [84]:
v = (0,None) # if the value in the column is 0, replace it with None (or another statistic like the z score or mean of that column...)
v[1]

In [85]:
df_filtered = df_replaced_values(df, value=v, numeric_features=numeric_features, quantile=None)

df_filtered.show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+



In [17]:
# replacement = replace(df_clean, "salary", -1, None)
# df_clean = df_clean.withColumn("salary", F.lit(replacement))

In [19]:
F.whe

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|     0|
+---------+----------+--------+-----+------+------+

